In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Load in CSV data files - Regular and Playoffs Games

    - Join the two data sets to create on large dataframe

In [3]:
file_path = Path('../Resources')

# Load the regular season data
reg_df = pd.read_csv(f"{file_path}/raptors_regulars.csv")

# Load the playoff season data
playoff_df = pd.read_csv(f"{file_path}/raptorsplayoffs.csv")


In [4]:
# Inspect columns and data types for data

print(reg_df.dtypes)

print(playoff_df.dtypes)

TEAM        object
DATE        object
MATCHUP     object
W/L         object
MIN          int64
PTS          int64
FGM          int64
FGA          int64
FG%        float64
3PM          int64
3PA          int64
3P%        float64
FTM          int64
FTA          int64
FT%        float64
OREB         int64
DREB         int64
REB          int64
AST          int64
STL          int64
BLK          int64
TOV          int64
PF           int64
+/-          int64
dtype: object
TEAM        object
DATE        object
MATCHUP     object
W/L         object
MIN          int64
PTS          int64
FGM          int64
FGA          int64
FG%        float64
3PM          int64
3PA          int64
3P%        float64
FTM          int64
FTA          int64
FT%        float64
OREB         int64
DREB         int64
REB          int64
AST          int64
STL          int64
BLK          int64
TOV          int64
PF           int64
+/-          int64
dtype: object


In [5]:
# Join the 2 datasets using concat()

joined_df = pd.concat([reg_df,playoff_df], ignore_index=True)

joined_df.head(3)

,TEAM,DATE,MATCHUP,W/L,MIN,PTS,FGM,FGA,FG%,3PM,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,TOR,01/13/2019,TOR @ WAS,W,290,140,49,104,47.1,13,...,80.6,17,42,59,24,16,8,21,27,2
1,TOR,11/29/2018,TOR vs. GSW,W,265,131,47,90,52.2,15,...,91.7,7,30,37,25,5,6,14,23,3
2,TOR,02/13/2019,TOR vs. WAS,W,241,129,44,92,47.8,16,...,83.3,9,43,52,32,8,8,14,27,9


# Clean the data

 - Drop TEAM, MATCHUP, DATE, and Score (+/-), 
     - We wont be using the columns as features in the ML model
 - Convert W/L columnn to integer datatype (Target Column)
     - W is set to = 0 and L is set to = 1
 - Drop any null data

In [6]:
# Drop columns that will not be used for ML models
# Drop TEAM, MATCHUP, and Score (+/-) for reg df
dropped_df = joined_df.drop(["DATE", "TEAM","MATCHUP","+/-"], axis=1)

dropped_df.head(3)

,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
0,W,290,140,49,104,47.1,13,36,36.1,29,36,80.6,17,42,59,24,16,8,21,27
1,W,265,131,47,90,52.2,15,38,39.5,22,24,91.7,7,30,37,25,5,6,14,23
2,W,241,129,44,92,47.8,16,38,42.1,25,30,83.3,9,43,52,32,8,8,14,27


In [7]:
# Check column data types
dropped_df.dtypes

W/L      object
MIN       int64
PTS       int64
FGM       int64
FGA       int64
FG%     float64
3PM       int64
3PA       int64
3P%     float64
FTM       int64
FTA       int64
FT%     float64
OREB      int64
DREB      int64
REB       int64
AST       int64
STL       int64
BLK       int64
TOV       int64
PF        int64
dtype: object

In [8]:
# Count the number of wins and losses
dropped_df["W/L"].value_counts()

W    74
L    32
Name: W/L, dtype: int64

In [9]:
# Convert W/L column 
# Make W = 0 and L = 1
# Count the number of wins and losses to confirm conversion
# Convert object type to int64 datatype

dropped_df.loc[dropped_df['W/L'] == "W", "W/L"] = 0
dropped_df.loc[dropped_df['W/L'] == "L", "W/L"] = 1

dropped_df["W/L"] = pd.to_numeric(dropped_df["W/L"])
    
dropped_df["W/L"].value_counts()

0    74
1    32
Name: W/L, dtype: int64

In [10]:
# Check column data types
dropped_df.dtypes

W/L       int64
MIN       int64
PTS       int64
FGM       int64
FGA       int64
FG%     float64
3PM       int64
3PA       int64
3P%     float64
FTM       int64
FTA       int64
FT%     float64
OREB      int64
DREB      int64
REB       int64
AST       int64
STL       int64
BLK       int64
TOV       int64
PF        int64
dtype: object

In [11]:
# Drop null columns and rows

# Drop null columns
cleaned_df = dropped_df.dropna(axis='columns', how='all')

# Drop null rows
cleaned_df = cleaned_df.dropna()

cleaned_df.head(3)

,W/L,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
0,0,290,140,49,104,47.1,13,36,36.1,29,36,80.6,17,42,59,24,16,8,21,27
1,0,265,131,47,90,52.2,15,38,39.5,22,24,91.7,7,30,37,25,5,6,14,23
2,0,241,129,44,92,47.8,16,38,42.1,25,30,83.3,9,43,52,32,8,8,14,27


# Split the Data into Training and Testing

In [12]:
# Create our features

X = cleaned_df.copy()
X = X.drop("W/L", axis = 1)

X.head(3)

,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
0,290,140,49,104,47.1,13,36,36.1,29,36,80.6,17,42,59,24,16,8,21,27
1,265,131,47,90,52.2,15,38,39.5,22,24,91.7,7,30,37,25,5,6,14,23
2,241,129,44,92,47.8,16,38,42.1,25,30,83.3,9,43,52,32,8,8,14,27


In [13]:
# Create our target

target = ["W/L"]

y = cleaned_df[target]
y[:5]

,W/L
0,0
1,0
2,0
3,0
4,0


In [14]:
X.describe()

,MIN,PTS,FGM,FGA,FG%,3PM,3PA,3P%,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF
count,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000,106.000000
mean,242.349057,112.660377,41.169811,88.103774,46.934906,12.273585,33.943396,36.039623,18.047170,22.283019,81.386792,9.292453,35.254717,44.547170,24.773585,8.235849,5.160377,12.915094,21.386792
std,8.866247,11.018501,4.705562,6.883976,5.795648,3.511974,5.366262,8.347383,5.677662,6.974450,9.092614,3.799667,4.612739,5.648653,5.368840,3.302386,2.398955,3.532483,3.951060
min,238.000000,86.000000,28.000000,69.000000,29.500000,4.000000,20.000000,20.000000,6.000000,7.000000,58.800000,2.000000,25.000000,31.000000,13.000000,1.000000,0.000000,6.000000,13.000000
25%,239.000000,105.000000,38.000000,84.000000,42.700000,10.000000,30.000000,29.000000,13.250000,17.000000,76.375000,6.000000,32.250000,41.000000,21.000000,6.000000,4.000000,10.000000,19.000000
50%,240.000000,114.000000,41.000000,88.500000,47.350000,12.000000,34.000000,35.800000,18.000000,22.000000,81.550000,8.000000,35.000000,44.000000,25.000000,8.000000,5.000000,13.000000,21.000000
75%,241.000000,121.750000,44.000000,92.000000,51.575000,15.000000,38.000000,41.450000,22.000000,27.000000,87.300000,12.000000,38.000000,49.000000,29.000000,10.000000,6.000000,15.000000,24.000000
max,290.000000,140.000000,53.000000,105.000000,60.900000,21.000000,47.000000,58.100000,32.000000,40.000000,100.000000,18.000000,47.000000,59.000000,36.000000,19.000000,12.000000,23.000000,30.000000


In [15]:
# Check the balance of our target values

y["W/L"].value_counts()

0    74
1    32
Name: W/L, dtype: int64

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Oversampling

### Naive Random Oversampling

In [17]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'W/L': 1})

In [18]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.8583333333333334

In [20]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index = ["Actual Wins", "Actual Losses"], columns=["Predicted Wins", "Predicted Losses"])
cm_df

,Predicted Wins,Predicted Losses
Actual Wins,12,3
Actual Losses,1,11


In [21]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.80      0.92      0.86      0.86      0.72        15
          1       0.79      0.92      0.80      0.85      0.86      0.74        12

avg / total       0.86      0.85      0.86      0.85      0.86      0.73        27



### SMOTE Oversampling

In [22]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'W/L': 1})

In [23]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [24]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.8

In [25]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index = ["Actual Wins", "Actual Losses"], columns=["Predicted Wins", "Predicted Losses"])
cm_df

,Predicted Wins,Predicted Losses
Actual Wins,14,1
Actual Losses,4,8


In [26]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.78      0.93      0.67      0.85      0.79      0.64        15
          1       0.89      0.67      0.93      0.76      0.79      0.61        12

avg / total       0.83      0.81      0.79      0.81      0.79      0.62        27



# Undersampling

### ClusterCentroids

In [27]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'W/L': 1})

In [28]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [29]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.8500000000000001

In [30]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index = ["Actual Wins", "Actual Losses"], columns=["Predicted Wins", "Predicted Losses"])
cm_df

,Predicted Wins,Predicted Losses
Actual Wins,13,2
Actual Losses,2,10


In [31]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.87      0.87      0.83      0.87      0.85      0.72        15
          1       0.83      0.83      0.87      0.83      0.85      0.72        12

avg / total       0.85      0.85      0.85      0.85      0.85      0.72        27



# Combination (Over and Under) Sampling

### SMOTTEN

In [32]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
Counter(y_resampled)

Counter({'W/L': 1})

In [33]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [34]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.8

In [35]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index = ["Actual Wins", "Actual Losses"], columns=["Predicted Wins", "Predicted Losses"])
cm_df

,Predicted Wins,Predicted Losses
Actual Wins,9,6
Actual Losses,0,12


In [36]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       1.00      0.60      1.00      0.75      0.77      0.58        15
          1       0.67      1.00      0.60      0.80      0.77      0.62        12

avg / total       0.85      0.78      0.82      0.77      0.77      0.60        27



# Ensemble Learners

### Balanced Random Forest Classifier

In [37]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
predictions = rf_model.predict(X_test)
print(Counter(y_train['W/L']))

Counter({0: 59, 1: 20})


In [38]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, predictions)

0.75

In [39]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(
    cm, index = ["Actual Wins", "Actual Losses"], columns=["Predicted Wins", "Predicted Losses"])
cm_df

,Predicted Wins,Predicted Losses
Actual Wins,10,5
Actual Losses,2,10


In [40]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.83      0.67      0.83      0.74      0.75      0.55        15
          1       0.67      0.83      0.67      0.74      0.75      0.56        12

avg / total       0.76      0.74      0.76      0.74      0.75      0.55        27



In [41]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_

sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.12940304877653544, 'FG%'),
 (0.125034243025812, 'PTS'),
 (0.10934654452271303, '3P%'),
 (0.07454862190529676, 'AST'),
 (0.06867357653040207, 'STL'),
 (0.058700609091950656, '3PM'),
 (0.052916041157205905, 'FGM'),
 (0.052502988943708924, 'DREB'),
 (0.04987324832245125, 'FT%'),
 (0.04197031934984737, 'FTM'),
 (0.033130620963574026, 'FGA'),
 (0.032839124047903775, 'OREB'),
 (0.031270145866015224, 'REB'),
 (0.027633964587054527, 'BLK'),
 (0.027513642243011484, 'TOV'),
 (0.024248147451897222, 'FTA'),
 (0.02308717690507341, '3PA'),
 (0.021069537107568048, 'MIN'),
 (0.01623839920197904, 'PF')]

### Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model.fit(X_train, y_train)
predictions = ee_model.predict(X_test)
print(Counter(y_train['W/L']))

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, predictions)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predictions)

cm_df = pd.DataFrame(
    cm, index = ["Actual Wins", "Actual Losses"], columns=["Predicted Wins", "Predicted Losses"])
cm_df

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))